In [ ]:
"""
Prepare cav mean and cav std

"""

In [ ]:
import xarray as xr
import basal_melt_param.useful_functions as uf
from tqdm.notebook import tqdm

READ IN DATA

In [ ]:
mod = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'
to2300 = False

if scenario == 'historical':
    yystart = 1850
    yyend = 2014
else:
    if to 2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
inputpath_mask='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachine_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

for tt in tqdm(range(yy_start, yy_end+1)): #continue at 1978

    # T and S extrapolated to ice draft depth
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')

    n = 0
    for kisf in file_isf.Nisf:
        #conc_isf = uf.choose_isf(file_conc_cut['isfdraft_conc'],isf_stack_mask, kisf)
        T_isf = uf.choose_isf(T_S_2D_isfdraft['theta_in'],isf_stack_mask, kisf)
        S_isf = uf.choose_isf(T_S_2D_isfdraft['salinity_in'],isf_stack_mask, kisf)

        T_mean_cav = T_isf.mean('mask_coord').to_dataset(name='T_mean')
        S_mean_cav = S_isf.mean('mask_coord').to_dataset(name='S_mean')
        T_std_cav = T_isf.std('mask_coord').to_dataset(name='T_std')
        S_std_cav = S_isf.std('mask_coord').to_dataset(name='S_std')

        all_TS_isf = xr.merge([T_mean_cav,S_mean_cav,T_std_cav,S_std_cav])
        all_TS_isf_2D = all_TS_isf.where(T_isf).load()

        if n == 0:
            all_TS_all = all_TS_isf_2D.squeeze().drop('Nisf')
        else:
             all_TS_all =  all_TS_all.combine_first(all_TS_isf_2D).squeeze().drop('Nisf')
        n = n+1

        del all_TS_isf_2D

    all_TS_all_unstacked = uf.bring_back_to_2D(all_TS_all)
    
    all_TS_all_unstacked.to_netcdf(inputpath_profiles + 'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'.nc')

In [ ]:
print('test')